## Methods Used

Two most common types of recommender systems are **Content-Based** and **Collaborative Filtering (CF)**. 

* Collaborative filtering produces recommendations based on the knowledge of users’ attitude to items, that is it uses the "wisdom of the crowd" to recommend items. 
* Content-based recommender systems focus on the attributes of the items and give you recommendations based on the similarity between them.


## Collaborative Filtering

In general, Collaborative filtering (CF) is more commonly used than content-based systems because it usually gives better results and is relatively easy to understand (from an overall implementation perspective). The algorithm has the ability to do feature learning on its own, which means that it can start to learn for itself what features to use. 

CF can be divided into **Memory-Based Collaborative Filtering** and **Model-Based Collaborative filtering**. 

we will implement and Memory-Based CF by computing cosine similarity. 

In [1]:
import numpy as np
import pandas as pd

In [3]:
df1 = pd.read_excel('cleaned_data1.xlsx')
display(df1.head(2) )
df2 = pd.read_excel('cleaned_data2.xlsx')
display(df2.head(2) )

,Song,use_count,Name,Followers,Posts,Bio,URL,ER,title,release,artist_name,year,Delta,Curr_ER,Comments,Likes,Time
0,SOAKIMP12A8C130995,1,tejaswini wagh,83789,348,“ You'll never find peace of mind until you li...,https://www.instagram.com/waghtejaswini/,9.9,The Cove,Thicker Than Water,Jack Johnson,0,-7.719191,2.180809,1827,16445,3d
1,SOAKIMP12A8C130995,1,Shounak Nayak,7151,505,UK/India All posts my own unless stated. Email:,https://www.instagram.com/shounaknayak/,3.4,The Cove,Thicker Than Water,Jack Johnson,0,1.337458,4.737458,161,967,1d


,Song,use_count,Name,Followers,Posts,Bio,URL,ER,title,release,artist_name,year,Delta,Curr_ER,Comments,Likes,Time
0,SOPZKGR12A6D4F3F3A,1,RIDHIMA,3322,400,Wooden works by hand Shop: +998 97 543 0000 Ma...,https://www.instagram.com/ridz5014/,0.7,Red Red Wine (Edit),Original Hits - Party,UB40,2008,4.623247,5.323247,2750,13754,4w
1,SOPZKGR12A6D4F3F3A,1,None,26241,912,Si vede bene solo con il cuore. L'essenziale è...,https://www.instagram.com/lady_golf_mk4/,0.3,Red Red Wine (Edit),Original Hits - Party,UB40,2008,3.376269,3.676269,9325,83928,4w


In [4]:
#append two dataset
df = df1.append(df2, ignore_index=False)
df.tail(2)

,Song,use_count,Name,Followers,Posts,Bio,URL,ER,title,release,artist_name,year,Delta,Curr_ER,Comments,Likes,Time
44904,SOGCHYZ12AF72A69EC,2,Riya Deepsi,25955,515,Human being Email for any collaboration or enq...,https://www.instagram.com/riya_d_m/,3.2,That Tree (feat. Kid Cudi),That Tree Featuring Kid Cudi,Snoop Dogg featuring Kid Cudi,2010,-0.05021,3.14979,7721,38605,2w
44905,SOSSZPW12A8C13843D,20,Kashika Kapur Makeupartist,49130,1921,Dreamer of making this world a better place🙏🏻 ...,https://www.instagram.com/kashikakapurmua/,2.9,Figures,Dreams,The Whitest Boy Alive,2006,-2.33451,0.56549,92,833,1d


In [26]:
n_users = df.URL.nunique()
n_items = df.Song.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Songs: '+str(n_items))

Num. of Users: 4218
Num of Songs: 9921


In [31]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.25)

## Memory-Based Collaborative Filtering

Memory-Based Collaborative Filtering approaches can be divided into two main sections: **user-item filtering** and **item-item filtering**. 

A *user-item filtering* will take a particular user, find users that are similar to that user based on similarity of ratings, and recommend items that those similar users liked. 

In contrast, *item-item filtering* will take an item, find users who liked that item, and find other items that those users or similar users also liked. It takes items and outputs other items as recommendations. 

* *Item-Item Collaborative Filtering*: “Users who liked this song also liked …”
* *User-Item Collaborative Filtering*: “Users who are similar to you also liked …”

In both cases, you create a user-item matrix which built from the entire dataset.

Since we have split the data into testing and training we will need to create two ``[users x songs]`` matrices (all users by all songs). 

The training matrix contains 75% of the ratings and the testing matrix contains 25% of the ratings.  

* After you have built the user-item matrix you calculate the similarity and create a similarity matrix. 

The similarity values between items in *Item-Item Collaborative Filtering* are measured by observing all the users who have rated both items.  

* For *User-Item Collaborative Filtering* the similarity values between users are measured by observing all the items that are rated by both users.

In [11]:
#Let us create Ids for user

user_codes = df.URL.drop_duplicates().reset_index()
user_codes.rename(columns={'index':'user_old_index'}, inplace=True)
user_codes['user_new_index'] = list(user_codes.index)
user_codes.head(3)

,user_old_index,URL,user_new_index
0,0,https://www.instagram.com/waghtejaswini/,0
1,1,https://www.instagram.com/shounaknayak/,1
2,2,https://www.instagram.com/jdinstitute/,2


In [12]:
#Let us create Ids for Item

song_codes = df.Song.drop_duplicates().reset_index()
song_codes.rename(columns={'index':'song_old_index'}, inplace=True)
song_codes['song_new_index'] = list(song_codes.index)
song_codes.head(7)

,song_old_index,Song,song_new_index
0,0,SOAKIMP12A8C130995,0
1,9,SOBBMDR12A8C13253B,1
2,14,SOBXHDL12A81C204C0,2
3,62,SOBYHAJ12A6701BF1D,3
4,89,SODACBL12A8C13C273,4
5,134,SODDNQT12A6D4F5F7E,5
6,140,SODXRTY12AB0180F3B,6


In [13]:
# Merge song ids, and User ids

df = pd.merge(df, song_codes,how='left')
df = pd.merge(df, user_codes,how='left')
df.head(3)

,Song,use_count,Name,Followers,Posts,Bio,URL,ER,title,release,...,year,Delta,Curr_ER,Comments,Likes,Time,song_old_index,song_new_index,user_old_index,user_new_index
0,SOAKIMP12A8C130995,1,tejaswini wagh,83789,348,“ You'll never find peace of mind until you li...,https://www.instagram.com/waghtejaswini/,9.9,The Cove,Thicker Than Water,...,0,-7.719191,2.180809,1827,16445,3d,0,0,0,0
1,SOAKIMP12A8C130995,1,Shounak Nayak,7151,505,UK/India All posts my own unless stated. Email:,https://www.instagram.com/shounaknayak/,3.4,The Cove,Thicker Than Water,...,0,1.337458,4.737458,161,967,1d,0,0,1,1
2,SOAKIMP12A8C130995,3,JD Institute of Fashion,16047,3120,"Empowering creative minds since 1988, JD insti...",https://www.instagram.com/jdinstitute/,0.5,The Cove,Thicker Than Water,...,0,0.130806,0.630806,1012,9110,4w,0,0,2,2


A distance metric commonly used in recommender systems is *cosine similarity*, where the ratings are seen as vectors in ``n``-dimensional space and the similarity is calculated based on the angle between these vectors. 
Cosine similiarity for users *a* and *m* can be calculated using the formula below, where you take dot product of  the user vector *$u_k$* and the user vector *$u_a$* and divide it by multiplication of the Euclidean lengths of the vectors.
<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(u_k,u_a)=\frac{u_k&space;\cdot&space;u_a&space;}{&space;\left&space;\|&space;u_k&space;\right&space;\|&space;\left&space;\|&space;u_a&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{k,m}x_{a,m}}{\sqrt{\sum&space;x_{k,m}^2\sum&space;x_{a,m}^2}}"/>

To calculate similarity between items *m* and *b* you use the formula:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?s_u^{cos}(i_m,i_b)=\frac{i_m&space;\cdot&space;i_b&space;}{&space;\left&space;\|&space;i_m&space;\right&space;\|&space;\left&space;\|&space;i_b&space;\right&space;\|&space;}&space;=\frac{\sum&space;x_{a,m}x_{a,b}}{\sqrt{\sum&space;x_{a,m}^2\sum&space;x_{a,b}^2}}
"/>

Your first step will be to create the user-item matrix. Since you have both testing and training data you need to create two matrices.  

In [ ]:
#Create two user-item matrices, one for training and another for testing

train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[21]-1, line[19]-1] = line[13] 

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[21]-1, line[19]-1] = line[13]

You can use the [pairwise_distances](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html) function from sklearn to calculate the cosine similarity.

In [34]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

Next step is to make predictions. You have already created similarity matrices: `user_similarity` and `item_similarity` and therefore you can make a prediction by applying following formula for user-based CF:

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\bar{x}_{k}&space;&plus;&space;\frac{\sum\limits_{u_a}&space;sim_u(u_k,&space;u_a)&space;(x_{a,m}&space;-&space;\bar{x_{u_a}})}{\sum\limits_{u_a}|sim_u(u_k,&space;u_a)|}"/>

You can look at the similarity between users *k* and *a* as weights that are multiplied by the Delta of a similar user *a* (corrected for the average Delta of that user). You will need to normalize it so that the ratings stay between 1 and 5 and, as a final step, sum the average ratings for the user that you are trying to predict. 

The idea here is that some users may tend always to give high or low Delta to all songs as their followers crowds are. The relative difference in the Delta that these users get is more important than the absolute values. To give an example: suppose, user *k* gives 4 stars to his favourite movies and 3 stars to all other good movies. Suppose now that another user *t* rates movies that he/she likes with 5 stars, and the movies he/she fell asleep over with 3 stars. These two users could have a very similar taste but treat the rating system differently. 

When making a prediction for item-based CF you don't need to correct for users average rating since query user itself is used to do predictions.

<img class="aligncenter size-thumbnail img-responsive" src="https://latex.codecogs.com/gif.latex?\hat{x}_{k,m}&space;=&space;\frac{\sum\limits_{i_b}&space;sim_i(i_m,&space;i_b)&space;(x_{k,b})&space;}{\sum\limits_{i_b}|sim_i(i_m,&space;i_b)|}"/>

In [35]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [36]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

### Evaluation
There are many evaluation metrics but one of the most popular metric used to evaluate accuracy of predicted ratings is *Root Mean Squared Error (RMSE)*. 
<img src="https://latex.codecogs.com/gif.latex?RMSE&space;=\sqrt{\frac{1}{N}&space;\sum&space;(x_i&space;-\hat{x_i})^2}" title="RMSE =\sqrt{\frac{1}{N} \sum (x_i -\hat{x_i})^2}" />
​
You can use the [mean_square_error](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) (MSE) function from `sklearn`, where the RMSE is just the square root of MSE. To read more about different evaluation metrics you can take a look at [this article](http://research.microsoft.com/pubs/115396/EvaluationMetrics.TR.pdf). 

Since you only want to consider predicted ratings that are in the test dataset, you filter out all other elements in the prediction matrix with `prediction[ground_truth.nonzero()]`. 

In [37]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

In [38]:
print('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))

User-based CF RMSE: 2.8828404060427064
Item-based CF RMSE: 2.8853598092386368


In [ ]:
# So Far the results have been good.